## Text input

https://platform.openai.com/docs/models

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent

model = ChatOllama(model="gpt-oss:20b", temperature=0.8)

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content=[
    {"type": "text", "text": "What is the capital of The Moon?"}
])

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

**The capital of the Moon is called *Luna Nova*—the “New Moon City.”**

Nestled in the heart of the Mare Tranquillitatis, Luna Nova rises from the regolith as a gleaming lattice of white and silvery titanium. Its concentric rings of habitat modules form a ringed city that orbits the surface like a living, breathing organism. At its core sits the *Heliarch*—a transparent, rotating dome that houses the Lunar Council and the Interplanetary Assembly, where representatives from Earth, Mars, and the asteroid belt convene to govern the Moon’s colonies.

Key features of Luna Nova:

- **Solar Nexus** – A vast photovoltaic array that powers the entire city, feeding energy into the lunar grid and providing excess power to off‑world missions.
- **The Lattice Market** – A bustling marketplace built into the city’s outer rings, where traders exchange Earth‑derived goods, lunar regolith‑based materials, and exotic bio‑engineered food.
- **The Lunar Library** – A repository of all human knowledge, sto

## Image input

In [4]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.jpg', multiple=False)
display(uploader)

FileUpload(value=(), accept='.jpg', description='Upload')

In [5]:
print(uploader.value)

({'name': 'langchain-picture.jpg', 'type': 'image/jpeg', 'size': 46865, 'content': <memory at 0x7c70147d9240>, 'last_modified': datetime.datetime(2026, 2, 6, 18, 13, 50, 575000, tzinfo=datetime.timezone.utc)},)


In [6]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [7]:
model = ChatOllama(model="qwen3-vl:8b", temperature=0.8)

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

In [ ]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this capital"},
    {"type": "image", "base64": img_b64, "mime_type": "image/.jpg"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

## Audio input

In [ ]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.mp3', multiple=False)
display(uploader)

In [ ]:
print(uploader.value)

In [ ]:
import base64
import io
from faster_whisper import WhisperModel
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# 1. Initialize the Transcription Model (Runs locally)
# "base" is fast, "small" is more accurate
stt_model = WhisperModel("base", device="cpu", compute_type="int8")

def get_text_from_base64(b64_string):
    # Decode base64 to bytes
    audio_data = base64.b64decode(b64_string)
    audio_file = io.BytesIO(audio_data)
    
    # Transcribe
    segments, _ = stt_model.transcribe(audio_file, beam_size=5)
    return " ".join([segment.text for segment in segments])

# 2. Get the text
transcribed_text = get_text_from_base64(aud_b64)

# 3. Pass to your Ollama Model
model = ChatOllama(model="qwen3-vl:8b", temperature=0.8)

# We send the transcription AS text because ChatOllama doesn't support the 'audio' block yet
response = model.invoke([
    HumanMessage(content=f"The user provided an audio file that says: '{transcribed_text}'. Respond to it.")
])

print(response.content)